In [4]:
import pandas as pd
df = pd.read_excel('v2_script_10/exp12/20240417_eval.xlsx', sheet_name=1)
df

,"train:test = 80:20, trials = 10",Unnamed: 1,Unnamed: 2,Org(1024),EncDec(256),nonlinear head(256),PCgrad(256),IRM(1024),PCgrad(1024),nonlinear head + separate step(256)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dataset,functional,N_samples,org,pca,9f,12c,11b,11c,9g
5,AraComputational2022,PBE-D3,64,0.31 ± 0.56,-0.21 ± 0.92,-0.16 ± 1.05,-0.08 ± 0.74,-0.18 ± 1.45,-0.68 ± 0.76,-0.01 ± 0.42
6,BajdichWO32018,PBE+U,84,0.62 ± 0.16,0.41 ± 0.11,0.44 ± 0.18,0.43 ± 0.17,-0.35 ± 0.51,-0.53 ± 0.39,0.35 ± 0.34
7,BoesAdsorption2018,RPBE,49,0.51 ± 0.30,-0.63 ± 1.84,0.26 ± 0.79,0.39 ± 0.33,0.11 ± 0.89,0.25 ± 0.63,0.06 ± 1.63
8,ComerUnraveling2022,PBE+U,66,0.59 ± 0.43,0.17 ± 0.60,0.31 ± 0.49,0.18 ± 0.42,0.15 ± 0.77,0.53 ± 0.32,0.27 ± 0.37
9,HossainInvestigation2022,PBE+U,365,0.80 ± 0.03,0.60 ± 0.13,0.73 ± 0.07,0.55 ± 0.09,0.76 ± 0.06,0.41 ± 0.07,0.68 ± 0.09


In [36]:
import os
import numpy as np
import torch
import pickle
import json
import pandas as pd
import torch.nn as nn
from torch.nn import DataParallel
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from itertools import cycle
import torch.autograd as autograd
from torch.nn.utils import parameters_to_vector, vector_to_parameters
from sklearn.decomposition import PCA
import torch.optim as optim
from torch.nn import DataParallel

In [37]:
seed = 42
ratio_test = 0.2
np.random.seed(seed)
torch.manual_seed(seed)
exp_name = 'exp_12c'
curdir = '' ## '/curdir/' ## ''

In [38]:
class Autoencoder(nn.Module):
    def __init__(self, len_emb=1024, abs_len_emb=256):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(len_emb, int(len_emb*0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(len_emb*0.5)),
            nn.Linear(int(len_emb*0.5), abs_len_emb)  # Bottleneck layer
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(abs_len_emb, int(len_emb*0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(len_emb*0.5)),
            nn.Linear(int(len_emb*0.5), len_emb),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss(reduction='mean')
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding
        self.encoder = nn.Sequential(
            nn.Linear(self.len_embedding, int(self.len_embedding*0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*0.5)),

            nn.Linear(int(self.len_embedding*0.5), self.abstract_len_embedding),
        )
        self.head1 = nn.Linear(self.abstract_len_embedding, 1)
        self.head2 = nn.Linear(self.abstract_len_embedding, 1)

    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)

In [39]:
list_pid = [
    'AraComputational2022',
    'BajdichWO32018',
    'BoesAdsorption2018',
    'ComerUnraveling2022',
    'HossainInvestigation2022',
    'KoshyInvestigation2022',
    'LiuCatalytic2022',
    'RaoResolving2022',
    'TettehCompressively2022',
    'WeiInsights2022',
    ]

In [40]:
for pid in list_pid[:]:
    list_maeo = []
    list_r2o = []
    list_maep = []
    list_r2p = []
    list_maei = []
    list_r2i = []
    
    for tno in range(10):

        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation original
        model_original = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_original.fit(X_train, y_train)
        y_pred_original = model_original.predict(X_test)
        mae_original = mean_absolute_error(y_test, y_pred_original)
        r2score_original = r2_score(y_test, y_pred_original)    

        ## evaluation pca
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        train_data = TensorDataset(X_train_tensor, X_train_tensor)  # Input is the same as target
        train_loader = DataLoader(train_data, batch_size=10000, shuffle=True)
        autoencoder = Autoencoder(len_emb=1024, abs_len_emb=256)
        optimizer = optim.Adam(autoencoder.parameters(), lr=1e-4)
        criterion = nn.L1Loss(reduction='mean')
        epochs = 10000
        for epoch in range(epochs):
            for data, target in train_loader:
                optimizer.zero_grad()
                pred = autoencoder(data)
                loss = criterion(pred, target)
                loss.backward()
                optimizer.step()
            if epoch%5000==0:
                print(f"Epoch {epoch+1}, Loss: {loss.item()}")
        # Use the encoder to transform the training and test data
        with torch.no_grad():
            X_train_pca = autoencoder.encoder(X_train_tensor)
            X_test_pca = autoencoder.encoder(X_test_tensor)
        X_train_pca = X_train_pca.numpy()
        X_test_pca = X_test_pca.numpy()
        model_pca = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_pca.fit(X_train_pca, y_train)
        y_pred_pca = model_pca.predict(X_test_pca)
        mae_pca = mean_absolute_error(y_test, y_pred_pca)
        r2score_pca = r2_score(y_test, y_pred_pca)    

        ## evaluation invariant
        len_embedding = 1024
        abstract_len_embedding = int(len_embedding*0.25)
        model_path = curdir + 'v2_script_10/exp12/model_invariant_' + exp_name + '.pt'
        model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant.eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maeo.append(mae_original)
        list_r2o.append(r2score_original)
        list_maep.append(mae_pca)
        list_r2p.append(r2score_pca)
        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maeo), np.std(list_maeo))
    print(np.mean(list_r2o), np.std(list_r2o))
    print(np.mean(list_maep), np.std(list_maep))
    print(np.mean(list_r2p), np.std(list_r2p))
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
Epoch 1, Loss: 569.558349609375
Epoch 5001, Loss: 514.48681640625
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
Epoch 1, Loss: 569.2937622070312
Epoch 5001, Loss: 514.2552490234375
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
Epoch 1, Loss: 568.9877319335938
Epoch 5001, Loss: 513.875
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
Epoch 1, Loss: 569.1917724609375
Epoch 5001, Loss: 514.1798706054688
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
Epoch 1, Loss: 569.4763793945312
Epoch 5001, Loss: 514.4011840820312
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
Epoch 1, Loss: 569.8125610351562
Epoch 5001, Loss: 514.8035278320312
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
Epoch 1, Loss: 568.6478881835938
Epoch 5001, Loss: 513.5564575195312
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
Epoch 1, L

In [41]:
## try old models

In [42]:
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss(reduction='mean')
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding
        self.encoder = nn.Sequential(
            nn.Linear(self.len_embedding, int(self.len_embedding*1)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*1)),

            nn.Linear(int(self.len_embedding*1), self.abstract_len_embedding),
        )
        self.head1 = nn.Linear(self.abstract_len_embedding, 1)
        self.head2 = nn.Linear(self.abstract_len_embedding, 1)

    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)
        
len_embedding = 1024
abstract_len_embedding = int(len_embedding*1)
model_path = curdir + 'v2_script_10/model_invariant_exp_11b.pt'

In [43]:
for pid in list_pid[:]:
    list_maei = []
    list_r2i = []    
    for tno in range(10):
    
        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation invariant
        model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant.eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
1.1479986335967676 0.1984122481376579
-0.17681220230381037 1.4532845177471967
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025)

In [44]:
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss(reduction='mean')
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding
        self.encoder = nn.Sequential(
            nn.Linear(self.len_embedding, int(self.len_embedding*1)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*1)),

            nn.Linear(int(self.len_embedding*1), self.abstract_len_embedding),
        )
        self.head1 = nn.Linear(self.abstract_len_embedding, 1)
        self.head2 = nn.Linear(self.abstract_len_embedding, 1)

    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)
        
len_embedding = 1024
abstract_len_embedding = int(len_embedding*1)
model_path = curdir + 'v2_script_10/model_invariant_exp_11c.pt'

In [45]:
for pid in list_pid[:]:
    list_maei = []
    list_r2i = []    
    for tno in range(10):
    
        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation invariant
        model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant.eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
1.5371081900728427 0.3977396771224058
-0.6752682870040198 0.7587750199690733
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025),

In [46]:
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss() 
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding  
        self.encoder = nn.Sequential(
            ## 1024 to ...
            nn.Linear(self.len_embedding, int(self.len_embedding*0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*0.5)), 
            ## ... to ...
            nn.Linear(int(self.len_embedding*0.5), self.abstract_len_embedding),
        )
        self.head1 = nn.Sequential(
            nn.Linear(self.abstract_len_embedding, int(self.abstract_len_embedding * 0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.abstract_len_embedding * 0.5)),  # Add BatchNorm1d here
            nn.Linear(int(self.abstract_len_embedding * 0.5), 1)
        )
        self.head2 = nn.Sequential(
            nn.Linear(self.abstract_len_embedding, int(self.abstract_len_embedding * 0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.abstract_len_embedding * 0.5)),  # Add BatchNorm1d here
            nn.Linear(int(self.abstract_len_embedding * 0.5), 1)
        )
        
    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)
        
len_embedding = 1024
abstract_len_embedding = int(len_embedding*0.25)
model_path = curdir + 'v2_script_10/model_invariant_exp9i.pt'

In [47]:
for pid in list_pid[:]:
    list_maei = []
    list_r2i = []    
    for tno in range(10):
    
        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation invariant
        model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant.eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
1.0479749359530188 0.1720054696549971
0.1941782828909441 0.5519110913594564
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), 

In [48]:
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss(reduction="mean") 
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding  
        self.encoder = nn.Sequential(
            ## 1024 to ...
            nn.Linear(self.len_embedding, int(self.len_embedding*0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*0.5)), 
            ## ... to ...
            nn.Linear(int(self.len_embedding*0.5), self.abstract_len_embedding),
        )
        self.head1 = nn.Sequential(
            nn.Linear(self.abstract_len_embedding, int(self.abstract_len_embedding * 0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.abstract_len_embedding * 0.5)),  # Add BatchNorm1d here
            nn.Linear(int(self.abstract_len_embedding * 0.5), 1)
        )
        self.head2 = nn.Sequential(
            nn.Linear(self.abstract_len_embedding, int(self.abstract_len_embedding * 0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.abstract_len_embedding * 0.5)),  # Add BatchNorm1d here
            nn.Linear(int(self.abstract_len_embedding * 0.5), 1)
        )
        
    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)
        
len_embedding = 1024
abstract_len_embedding = int(len_embedding*0.25)
model_path = curdir + 'v2_script_10/model_invariant_exp9h.pt'

In [49]:
for pid in list_pid[:]:
    list_maei = []
    list_r2i = []    
    for tno in range(10):

        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation invariant
        # model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant = DataParallel(InvariantModel(len_embedding, abstract_len_embedding))    
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant = model_invariant.module    
        model_invariant.train() ## eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
1.2507344166754328 0.28763506410644457
-0.011201906495235748 0.41778440056457933
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 10

In [50]:
## class
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss(reduction="mean") 
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding  
        self.encoder = nn.Sequential(
            ## 1024 to ...
            nn.Linear(self.len_embedding, int(self.len_embedding*0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*0.5)), 
            ## ... to ...
            nn.Linear(int(self.len_embedding*0.5), self.abstract_len_embedding),
        )
        self.head1 = nn.Sequential(
            nn.Linear(self.abstract_len_embedding, int(self.abstract_len_embedding * 0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.abstract_len_embedding * 0.5)),  # Add BatchNorm1d here
            nn.Linear(int(self.abstract_len_embedding * 0.5), 1)
        )
        self.head2 = nn.Sequential(
            nn.Linear(self.abstract_len_embedding, int(self.abstract_len_embedding * 0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.abstract_len_embedding * 0.5)),  # Add BatchNorm1d here
            nn.Linear(int(self.abstract_len_embedding * 0.5), 1)
        )
        
    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)
        
len_embedding = 1024
abstract_len_embedding = int(len_embedding*0.25)
model_path = curdir + 'v2_script_10/model_invariant_exp9f.pt'

In [51]:
for pid in list_pid[:]:
    list_maei = []
    list_r2i = []    
    for tno in range(10):

        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation invariant
        # model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant = DataParallel(InvariantModel(len_embedding, abstract_len_embedding))    
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant = model_invariant.module    
        model_invariant.train() ## eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
1.1215503688190902 0.20789104176487688
-0.16236661220683218 1.0452307280527935
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025

In [52]:
## class
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss(reduction="mean") 
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding  
        self.encoder = nn.Sequential(
            ## 1024 to ...
            nn.Linear(self.len_embedding, int(self.len_embedding*0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*0.5)), 
            ## ... to ...
            nn.Linear(int(self.len_embedding*0.5), self.abstract_len_embedding),
        )
        self.head1 = nn.Sequential(
            nn.Linear(self.abstract_len_embedding, int(self.abstract_len_embedding * 0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.abstract_len_embedding * 0.5)),  # Add BatchNorm1d here
            nn.Linear(int(self.abstract_len_embedding * 0.5), 1)
        )
        self.head2 = nn.Sequential(
            nn.Linear(self.abstract_len_embedding, int(self.abstract_len_embedding * 0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.abstract_len_embedding * 0.5)),  # Add BatchNorm1d here
            nn.Linear(int(self.abstract_len_embedding * 0.5), 1)
        )
        
    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)
        
len_embedding = 1024
abstract_len_embedding = int(len_embedding*0.25)
model_path = curdir + 'v2_script_10/model_invariant_exp9g.pt'

In [53]:
for pid in list_pid[:]:
    list_maei = []
    list_r2i = []    
    for tno in range(10):

        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation invariant
        # model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant = DataParallel(InvariantModel(len_embedding, abstract_len_embedding))    
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant = model_invariant.module    
        model_invariant.train() ## eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
1.2507344166754328 0.28763506410644457
-0.011201906495235748 0.41778440056457933
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 10

In [54]:
## class
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss(reduction="mean") 
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding  
        self.encoder = nn.Sequential(
            ## 1024 to ...
            nn.Linear(self.len_embedding, int(self.len_embedding*0.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*0.5)), 
            ## ... to ...
            nn.Linear(int(self.len_embedding*0.5), self.abstract_len_embedding),
        )        
        self.head1 = nn.Linear(self.abstract_len_embedding, 1)  # For dataset 1
        self.head2 = nn.Linear(self.abstract_len_embedding, 1)  # For dataset 2

    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)
        
len_embedding = 1024
abstract_len_embedding = int(len_embedding*0.25)
model_path = curdir + 'v2_script_10/model_invariant_exp9e.pt'

In [55]:
for pid in list_pid[:]:
    list_maei = []
    list_r2i = []    
    for tno in range(10):

        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation invariant
        # model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant = DataParallel(InvariantModel(len_embedding, abstract_len_embedding))    
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant = model_invariant.module    
        model_invariant.train() ## eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
1.3979544583288552 0.25000718858710347
-0.8057880291617348 2.2263920427489765
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025)

In [56]:
## class
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss(reduction="mean") 
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding  
        self.encoder = nn.Sequential(
            ## 1024 to ...
            nn.Linear(self.len_embedding, int(self.len_embedding)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding)), 
            ## ... to ...
            nn.Linear(int(self.len_embedding), self.abstract_len_embedding),
        )        
        self.head1 = nn.Linear(self.abstract_len_embedding, 1)  # For dataset 1
        self.head2 = nn.Linear(self.abstract_len_embedding, 1)  # For dataset 2

    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)
        
len_embedding = 1024
abstract_len_embedding = int(len_embedding*1)
model_path = curdir + 'v2_script_10/model_invariant_exp9d.pt'

In [57]:
for pid in list_pid[:]:
    list_maei = []
    list_r2i = []    
    for tno in range(10):

        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation invariant
        # model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant = DataParallel(InvariantModel(len_embedding, abstract_len_embedding))    
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant = model_invariant.module    
        model_invariant.train() ## eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
1.2939100129265326 0.19227543417680212
-0.2160203721183413 0.8214441907775833
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025)

In [58]:
## class
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss(reduction="mean") 
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding  
        self.encoder = nn.Sequential(
            ## 1024 to ...
            nn.Linear(self.len_embedding, int(self.len_embedding*2)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*2)), 
            ## ... to ...
            nn.Linear(int(self.len_embedding*2), int(self.len_embedding*1.5)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*1.5)),
            ## ... to ...
            nn.Linear(int(self.len_embedding*1.5), self.abstract_len_embedding),
        )        
        self.head1 = nn.Linear(self.abstract_len_embedding, 1)  # For dataset 1
        self.head2 = nn.Linear(self.abstract_len_embedding, 1)  # For dataset 2

    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)
        
len_embedding = 1024
abstract_len_embedding = int(len_embedding*1)
model_path = curdir + 'v2_script_10/model_invariant_exp9c.pt'

In [59]:
for pid in list_pid[:]:
    list_maei = []
    list_r2i = []    
    for tno in range(10):

        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation invariant
        # model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant = DataParallel(InvariantModel(len_embedding, abstract_len_embedding))    
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant = model_invariant.module    
        model_invariant.train() ## eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
1.2751408608681305 0.25528533357267547
-0.22496852022930636 0.9191321874133979
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025

In [60]:
## class
class InvariantModel(nn.Module):
    def __init__(self, len_embedding, abstract_len_embedding):
        super(InvariantModel, self).__init__()
        self.loss = nn.L1Loss(reduction="mean") 
        self.len_embedding = len_embedding
        self.abstract_len_embedding = abstract_len_embedding  
        self.encoder = nn.Sequential(
            ## 1024 to 2048
            nn.Linear(self.len_embedding, int(self.len_embedding*0.875)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*0.875)), 
            ## 2048 to 1536
            nn.Linear(int(self.len_embedding*0.875), int(self.len_embedding*0.75)),
            nn.ReLU(),
            nn.BatchNorm1d(int(self.len_embedding*0.75)),
            ## 1536 to 1024
            nn.Linear(int(self.len_embedding*0.75), self.abstract_len_embedding),
        )        
        self.head1 = nn.Linear(self.abstract_len_embedding, 1)  # For dataset 1
        self.head2 = nn.Linear(self.abstract_len_embedding, 1)  # For dataset 2

    def forward(self, x, dataset_id):
        x = self.encoder(x)
        if dataset_id == 1:
            return self.head1(x)
        else:
            return self.head2(x)
        
len_embedding = 1024
abstract_len_embedding = int(len_embedding*0.5)
model_path = curdir + 'v2_script_10/model_invariant_exp9.pt'

In [61]:
for pid in list_pid[:]:
    list_maei = []
    list_r2i = []    
    for tno in range(10):

        with open(f'{curdir}v2_script_10/exp12/df_{pid}.pickle', 'rb') as f:
            df = pickle.load(f)  
        with open(f'{curdir}v2_script_10/exp12/dict_{pid}.pickle', 'rb') as f:
            dinfo = pickle.load(f)          
        ##
        if int(df.shape[0]) < 0:
            continue
        ##
        print(f'pid: {pid}, df.shape:{df.shape}, dft_func: {dinfo[0]["dft_func"]}')    
        X = df.iloc[:, :-1].values
        y = df['nre'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=seed+tno)

        ## evaluation invariant
        # model_invariant = InvariantModel(len_embedding, abstract_len_embedding)
        model_invariant = DataParallel(InvariantModel(len_embedding, abstract_len_embedding))    
        model_invariant.load_state_dict(torch.load(model_path))
        model_invariant = model_invariant.module    
        model_invariant.train() ## eval()
        X_train_invariant = model_invariant.encoder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
        X_test_invariant = model_invariant.encoder(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
        model_invariant = xgb.XGBRegressor(learning_rate=0.2, max_depth=8, n_estimators=500)
        model_invariant.fit(X_train_invariant, y_train)
        y_pred_invariant = model_invariant.predict(X_test_invariant)
        mae_invariant = mean_absolute_error(y_test, y_pred_invariant)
        r2score_invariant = r2_score(y_test, y_pred_invariant)

        list_maei.append(mae_invariant)
        list_r2i.append(r2score_invariant)
        
    print(np.mean(list_maei), np.std(list_maei))
    print(np.mean(list_r2i), np.std(list_r2i))


pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
pid: AraComputational2022, df.shape:(64, 1025), dft_func: PBE-D3
1.3706716817751428 0.22315154980921315
-0.20644334093934275 0.7311316288749727
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025), dft_func: PBE+U
pid: BajdichWO32018, df.shape:(84, 1025